In [ ]:
#Import the libraries needed for Lending CLub Case study
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re as re


# fution to find the columns where all values are NaN
def find_column_with_all_NaN (df):
  nan_lst = [x for x in df.columns if len(df.index) == df[x].isnull().sum()]
  return nan_lst
       
#Deletes the given list of columns and modifies the dataframe accordingly
def del_columns_from_dataframe(df, del_lst) : 
  df.drop(del_lst, axis=1, inplace = True)

#Funtion to find the list of columns with more than/equlals to 90%  NaN values 
def find_column_with_90percent_or_more_NaN (df):
  list_90percent = [x for x in df.columns if((df[x].isnull().sum() / len(df.index))*100) >= 90]     
  return list_90percent

def convert_emp_len(emp_len) :
  emp_len = re.findall('(\d+)', emp_len) #emp_len.astype('str').str.extractall('(\d+)')
    
  #emp_len = emp_len.rstrip("+ years")
  #emp_len = emp_len.rstrip(" years")
  #emp_len = emp_len.rstrip(" year")
  #emp_len = emp_len.lstrip("<")
  #emp_len = [int(x) for x in emp_len.split() if x.isdigit()] 
  #print(emp_len)
  return(emp_len)

    
#Read the data set for the case study 
club_case_study_df = pd.read_csv('C:\\Users\\G50-70\\Downloads\\CaseStudys\\LendingClubCaseStudy\\loan\\loan.csv', low_memory=False)

#Print shape of dataframe to know number of columns and rows
print("Shape of dataframe : {}".format( club_case_study_df.shape) )

#print(club_case_study_df.head())


#print(club_case_study_df.describe())
#print(club_case_study_df.columns)

#Print number of missing values per column
print("Number of missing values per column : \n", club_case_study_df.isnull().sum(), "\n")

#From analysis missing values many columns have all missing values and find list of columns with NaN values
nan_column_lst = find_column_with_all_NaN(club_case_study_df)

#Drop the columans with all Nan
del_columns_from_dataframe(club_case_study_df, nan_column_lst)
print("Number of columns with all NaN vlaues : {}".format(len(nan_column_lst)))
print("Shape of data frame after dropping columnus with all NaN : {} \n".format(club_case_study_df.shape))

#Find columns with 90% or more of NaN values
list_90 = find_column_with_90percent_or_more_NaN(club_case_study_df)
#Delete columns with 90 0r more percent of NaN values
del_columns_from_dataframe(club_case_study_df, list_90)
print("Number of columns with all 90% or more NaN vlaues : {}".format(len(list_90)))
print("Shape of data frame after dropping columnus with 90% or more NaN : {} \n".format(club_case_study_df.shape))

#Print number of missing values per column after removing columns with more than 90% values missing
print("Number of missing values per column after droping columns : \n", club_case_study_df.isnull().sum(), "\n")

#print desc column to anylsis the data as this column has many missing valyes
print(club_case_study_df["desc"].head(20))
# desc column will not be used for loan decision also just description hence droping 
club_case_study_df.drop('desc', axis =1, inplace=True)

# Column 'mths_since_last_delinq' has more than 64% missing values. 
print("\nPrinting mths_since_last_delinq with non missing values")
print(club_case_study_df[club_case_study_df["mths_since_last_delinq"].isnull() == False]["mths_since_last_delinq"].tail(50) )
# Either we can drop or impute the missing values. 
#Decided to impute with zero for all mising values  

print("Number of missing values per column : \n", club_case_study_df.isnull().sum(), "\n")
print("Employee title , employee lenght \n", club_case_study_df[['emp_title', 'emp_length']].head())
#Drop row with emp__lengh as NaN
club_case_study_df = club_case_study_df[club_case_study_df['emp_length'].isnull() == False]
#Drop rows were emp_tite is NaN as it is different impute title and NaN are less
club_case_study_df = club_case_study_df[club_case_study_df['emp_title'].isnull() == False]
#print("Number of missing values per column : \n", club_case_study_df.isnull().sum(), "\n")

#print "collections_12_mths_ex_med" and "" columns for analysis
print(club_case_study_df[ ['collections_12_mths_ex_med','chargeoff_within_12_mths']  ].tail(50))
club_case_study_df = club_case_study_df[club_case_study_df['collections_12_mths_ex_med'].isnull() == False]
#print("Number of missing values per column : \n", club_case_study_df.isnull().sum(), "\n")

print(club_case_study_df['pub_rec_bankruptcies'][-150 : -100])

#Will impute missing values with zero as many values are zero
club_case_study_df['pub_rec_bankruptcies'].fillna(0, inplace = True)

#Fill mths_since_last_delinq with zero for missing value as many are missing.
club_case_study_df['mths_since_last_delinq'].fillna(0, inplace = True)

#Will drop rowa wherw  "last_pymnt_d" and "revol_util" are NaN as these have negligible in number 
club_case_study_df = club_case_study_df[club_case_study_df['last_pymnt_d'].isnull() == False]
club_case_study_df = club_case_study_df[club_case_study_df['revol_util'].isnull() == False]
print("Number of missing values per column : \n", club_case_study_df.isnull().sum(), "\n")


#Now we handled all missing values. Now will move analyze and correct data tupe of columns
#print(club_case_study_df.info())

# term, int_rate and zip_code should not be object. Let us inspect the values 
print("Term   and int_rate \n", club_case_study_df[['term', 'int_rate', 'zip_code']].head())
club_case_study_df['term'] = club_case_study_df['term'].apply(lambda x : int(x.rstrip(" months")))
club_case_study_df['int_rate'] = club_case_study_df['int_rate'].apply(lambda x : pd.to_numeric(x.rstrip("%")))
club_case_study_df['zip_code'] = club_case_study_df['zip_code'].apply(lambda x : int(x.rstrip("xx")))

# Looks grade, sub_grade, emp_lenght should not be objact, Print to analyze
print("Grade  Sub_grade  and emp_lenght\n", club_case_study_df[['grade', 'sub_grade', 'emp_length']].head())

#club_case_study_df['emp_length'] = club_case_study_df['emp_length'].apply(lambda x : convert_emp_len(x))
club_case_study_df['emp_length'] = club_case_study_df['emp_length'].apply(lambda x : re.findall('(\d+)' , str(x))[0])
club_case_study_df['emp_length'] = club_case_study_df['emp_length'].apply(lambda x : pd.to_numeric(x))

#Remove % from revol_util column
club_case_study_df['revol_util'] = club_case_study_df['revol_util'].apply(lambda x : pd.to_numeric(x.rstrip("%")))
#Print remaining columns which of object type for analysis
print(club_case_study_df[['earliest_cr_line','initial_list_status', 'application_type']].head())

print(club_case_study_df[club_case_study_df['initial_list_status']!='f']['initial_list_status'])

print(club_case_study_df.info())



#By looking at both columns data, both have NaN values in same rows, hence decided to drop 


#Univariate analysis 

#Plot bar plat on loan status , tearm, months , home_owership



#data_dictionary_df = pd.read_excel("C:\\Users\\G50-70\\Downloads\\CaseStudys\\LendingClubCaseStudy\\Data_Dictionary.xlsx")
#data_dictionary = data_dictionary_df.to_dict()
#print(data_dictionary)


Shape of dataframe : (39717, 111)
Number of missing values per column : 
 id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
                              ...  
tax_liens                        39
tot_hi_cred_lim               39717
total_bal_ex_mort             39717
total_bc_limit                39717
total_il_high_credit_limit    39717
Length: 111, dtype: int64 

Number of columns with all NaN vlaues : 54
Shape of data frame after dropping columnus with all NaN : (39717, 57) 

Number of columns with all 90% or more NaN vlaues : 2
Shape of data frame after dropping columnus with 90% or more NaN : (39717, 55) 

Number of missing values per column after droping columns : 
 id                                0
member_id                         0
loan_amnt                         0
funded_amnt                       0
funded_amnt_inv                   0
term     

Grade  Sub_grade  and emp_lenght
    grade sub_grade emp_length
1      C        C4   < 1 year
3      C        C1  10+ years
4      B        B5     1 year
5      A        A4    3 years
6      C        C5    8 years
7      E        E1    9 years
9      B        B5   < 1 year
10     C        C3    5 years
11     B        B5  10+ years
12     C        C1   < 1 year
13     B        B1    3 years
14     B        B2    3 years
15     D        D1   < 1 year
16     C        C4    4 years
17     A        A1  10+ years
18     B        B3     1 year
19     A        A1    6 years
20     C        C4    3 years
21     B        B4  10+ years
22     B        B3  10+ years
23     B        B3    5 years
24     B        B3     1 year
25     B        B1    2 years
27     D        D2    2 years
28     B        B3  10+ years
29     B        B3   < 1 year
31     A        A4    5 years
32     C        C5    2 years
33     A        A5    2 years
34     C        C5    7 years
35     B        B2  10+ years
36    